In [ ]:
%matplotlib inline
# packages to load 
# Check the versions of libraries
# Python version
import warnings
warnings.filterwarnings('ignore')
import sys
print('Python: {}'.format(sys.version))

import scipy
print('scipy: {}'.format(scipy.__version__))

import csv
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score

# Importing metrics for evaluation
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import random as rn
import math

from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn import svm
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix

from itertools import product

TRAIN_FILE_PATH = "data/X_train.csv"
TARGET_FILE_PATH =  "data/y_train.csv"
TEST_FILE_PATH = "data/X_test.csv"

seed=42
np.random.seed(seed)
rn.seed(seed)


In [27]:
#Load train and test set

train_data = pd.read_csv(TRAIN_FILE_PATH)
train_data.drop("id", axis=1, inplace=True)

Y_train = pd.read_csv(TARGET_FILE_PATH)
Y_train.drop("id", axis=1, inplace = True)

test_data =  pd.read_csv(TEST_FILE_PATH)
id_test = test_data.columns[0]
test_data.drop("id", axis=1, inplace=True)

In [3]:
def make_submission(filename, predictions):
    test_data =  pd.read_csv(TEST_FILE_PATH)
    test_data["y"] = predictions
    test_data[["id", "y"]].to_csv("submissions/"+filename, index= False)


def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

# SVM Overview

In [23]:
X_train = train_data.values
y_train = Y_train.values.ravel()


bmacs0 = []
bmacs1 = []
bmacs2 = []
bmacs_ensemble = []
bmacs_rfc = []

cmats0 = []
cmats1 = []
cmats2 = []
cmats_ensemble = []
cmats_rfc = []

n_samples = y_train.shape[0]
n_features = X_train.shape[1]
    
class_weight = n_samples / (3 * np.bincount((y_train).astype(int)))
class_weights0 = {
    0 : class_weight[0],
    1 : class_weight[1],
    2 : class_weight[2]
}

class_weights1 = { # tuned with bayesian optimization
    0: 3.2734,
    1: 0.6869,
    2: 2.6268
}

class_weights2 = { # tuned with bayesian optimization
    0 : 2.4720,
    1 : 0.4549,
    2 : 3.8704
}               

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
for train, test in kfold.split(X_train, y_train):
    
    X = X_train[train]
    y = y_train[train]
    
    clf0 = svm.SVC(class_weight=class_weights0)
    clf0.fit(X, y)
    
    clf1 = svm.SVC(class_weight=class_weights1)
    clf1.fit(X, y)
    
    clf2 = svm.SVC(class_weight=class_weights2)
    clf2.fit(X, y)
    
    clf_rfc = BalancedRandomForestClassifier(n_estimators=100, random_state=seed)
    clf_rfc.fit(X, y)
    
    eclf = VotingClassifier(estimators=[('clf0', clf0), ('clf1', clf1), ('clf2', clf2)], voting='hard')
    eclf.fit(X, y)
    
    X_valid = X_train[test]
    y_valid_true = y_train[test]
        
    y_valid_pred0 = clf0.predict(X_valid)
    y_valid_pred1 = clf1.predict(X_valid)
    y_valid_pred2 = clf2.predict(X_valid)
    y_valid_pred_ensemble = eclf.predict(X_valid)
    y_valid_pred_rfc = clf_rfc.predict(X_valid)

    cmat0 = confusion_matrix(y_valid_true, y_valid_pred0)
    bmac0 = balanced_accuracy_score(y_valid_true, y_valid_pred0)
    cmat1 = confusion_matrix(y_valid_true, y_valid_pred1)
    bmac1 = balanced_accuracy_score(y_valid_true, y_valid_pred1)
    cmat2 = confusion_matrix(y_valid_true, y_valid_pred2)
    bmac2 = balanced_accuracy_score(y_valid_true, y_valid_pred2)
    cmat_ensemble = confusion_matrix(y_valid_true, y_valid_pred_ensemble)
    bmac_ensemble = balanced_accuracy_score(y_valid_true, y_valid_pred_ensemble)
    cmat_rfc = confusion_matrix(y_valid_true, y_valid_pred_rfc)
    bmac_rfc = balanced_accuracy_score(y_valid_true, y_valid_pred_rfc)

    bmacs0.append(bmac0)
    bmacs1.append(bmac1)
    bmacs2.append(bmac2)
    bmacs_ensemble.append(bmac_ensemble)
    bmacs_rfc.append(bmac_rfc)
    
    print(f"bmac0={bmac0} bmac1={bmac1} bmac2={bmac2} bmacEnsemble={bmac_ensemble} bmacRfc={bmac_rfc}")
    
    cmats0.append(cmat0)
    cmats1.append(cmat1)
    cmats2.append(cmat2)
    cmats_ensemble.append(cmat_ensemble)
    cmats_rfc.append(cmat_rfc)
    
print(f"bmacs0: {bmacs0}")
print(f"bmacs1: {bmacs1}")
print(f"bmacs2: {bmacs2}")
print(f"bmacs_ensemble: {bmacs_ensemble}")
print(f"bmacs_rfc: {bmacs_rfc}")

print("AVG: BMAC0 score: %.4f (+/- %.4f)" % (np.mean(bmacs0), np.std(bmacs0)))
print("AVG: BMAC1 score: %.4f (+/- %.4f)" % (np.mean(bmacs1), np.std(bmacs1)))
print("AVG: BMAC2 score: %.4f (+/- %.4f)" % (np.mean(bmacs2), np.std(bmacs2)))
print("AVG: BMAC_ENSEMBLE score: %.4f (+/- %.4f)" % (np.mean(bmacs_ensemble), np.std(bmacs_ensemble)))
print("AVG: BMAC_RFC score: %.4f (+/- %.4f)" % (np.mean(bmacs_rfc), np.std(bmacs_rfc)))


bmac0=0.6907407407407408 bmac1=0.6638888888888889 bmac2=0.6722222222222222 bmacEnsemble=0.6925925925925926 bmacRfc=0.6712962962962963
bmac0=0.6749999999999999 bmac1=0.7074074074074074 bmac2=0.6749999999999999 bmacEnsemble=0.6777777777777777 bmacRfc=0.6731481481481482
bmac0=0.6916666666666668 bmac1=0.7074074074074074 bmac2=0.6953703703703704 bmacEnsemble=0.6916666666666668 bmacRfc=0.6592592592592592
bmac0=0.7398148148148148 bmac1=0.7138888888888889 bmac2=0.7305555555555556 bmacEnsemble=0.7490740740740741 bmacRfc=0.6749999999999999
bmac0=0.7425925925925926 bmac1=0.7481481481481481 bmac2=0.7175925925925926 bmacEnsemble=0.7462962962962963 bmacRfc=0.7064814814814815
bmac0=0.7324074074074075 bmac1=0.7101851851851851 bmac2=0.7416666666666666 bmacEnsemble=0.7351851851851853 bmacRfc=0.7111111111111111
bmac0=0.674074074074074 bmac1=0.6666666666666666 bmac2=0.6925925925925926 bmacEnsemble=0.6777777777777777 bmacRfc=0.65
bmac0=0.6898148148148149 bmac1=0.7083333333333334 bmac2=0.6768518518518518 bm

# Submission

In [31]:
X_train = train_data.values
y_train = Y_train.values.ravel()
X_test = test_data.values

class_weight = n_samples / (3 * np.bincount((y_train).astype(int)))
class_weights0 = { # svc defaults: Cross Validation: BMAC: 0.6992 (+/- 0.0314)
    0 : class_weight[0],
    1 : class_weight[1],
    2 : class_weight[2]
}

class_weights1 = { # tuned with bayesian optimization: Cross Validation: BMAC 0.7004 (+/- 0.0305)
    0: 3.2734,
    1: 0.6869,
    2: 2.6268
}

class_weights2 = { # tuned with bayesian optimization: Cross Validation:  BMAC 0.7006 (+/- 0.0248)
    0 : 2.4720,
    1 : 0.4549,
    2 : 3.8704
}  

clf0 = svm.SVC(class_weight=class_weights0)
clf1 = svm.SVC(class_weight=class_weights1)
clf2 = svm.SVC(class_weight=class_weights2)
eclf = VotingClassifier(estimators=[('clf0', clf0), ('clf1', clf1), ('clf2', clf2)], voting='hard')

eclf.fit(X_train, y_train)
pred = eclf.predict(X_test)

#  BMAC_ENSEMBLE score: 0.7018 std: 0.0328
make_submission('nk_ensemble_svm_with different_weights.csv', pred)

In [32]:
X_train = train_data.values
y_train = Y_train.values.ravel()
X_test = test_data.values


class_weights2 = { # tuned with bayesian optimization:  BMAC 0.7006 (+/- 0.0248)
    0 : 2.4720,
    1 : 0.4549,
    2 : 3.8704
}  

clf2 = svm.SVC(class_weight=class_weights2)
clf2.fit(X_train, y_train)
pred = clf2.predict(X_test)
# score: 0.7005 std: 0.0249
make_submission('nk_svm_bayesopt_weights_BMAC2.csv', pred)

# Bayesian Optimization

In [20]:
X_train = train_data.values
y_train = Y_train.values.ravel()


n_samples = y_train.shape[0]
n_features = X_train.shape[1]

def svm_cv(c0_weight=None, c1_weight=None, c2_weight=None, gamma=None):
    
    BMAC = []

    if gamma is None:
        1.0/n_features

    if c0_weight is not None and c1_weight is not None and c2_weight is not None:
        class_weights = {
            0 : c0_weight,
            1 : c1_weight,
            2 : c2_weight
        }
    else:
        #class_weight = n_samples / (3 * np.bincount((y_train).astype(int)))
        class_weights = { # tuned with bayesian optimization
            0 : 2.4720,
            1 : 0.4549,
            2 : 3.8704
        }
         
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
    for train, test in kfold.split(X_train, y_train):
        X = X_train[train]
        y = y_train[train]
    
        clf = svm.SVC(class_weight=class_weights, gamma=gamma)
        clf.fit(X, y)
    
        X_valid = X_train[test]
        y_valid_true = y_train[test]
        
        y_valid_pred = clf.predict(X_valid)

        cmat = confusion_matrix(y_valid_true, y_valid_pred)
        bmac = balanced_accuracy_score(y_valid_true, y_valid_pred)
 
        BMAC.append(bmac)
    
    print(f"bmacs: {BMAC}")
    print("AVG: BMAC score: %.4f (+/- %.4f)" % (np.mean(BMAC), np.std(BMAC)))
    return np.mean(BMAC)
    

### Bayesian Optimization: Train Class Weights

In [9]:
# Bayesian Optimization
from bayes_opt import BayesianOptimization

BO = BayesianOptimization(
        svm_cv,
        {'c0_weight': (0.5, 6), 
         'c1_weight': (0.1, 2), 
         'c2_weight': (0.5, 6)},
      random_state=seed
)
BO.explore({'c0_weight': [2.6666666666666665], 
              'c1_weight':[0.4444444444444444], 
              'c2_weight':[2.666666666666666],
            })

gp_params = {'alpha': 1e-5}

BO.maximize(n_iter=200, **gp_params)
  
print(BO.res['max'])
print(BO.res['all'])

Initialization
-----------------------------------------------------------------------
 Step |   Time |      Value |   c0_weight |   c1_weight |   c2_weight | 
bmacs: [0.6907407407407408, 0.6749999999999999, 0.6916666666666668, 0.7398148148148148, 0.7425925925925926, 0.7324074074074075, 0.674074074074074, 0.6898148148148149, 0.7157407407407407, 0.6398148148148148]
AVG: BMAC score: 0.6992 (+/- 0.0314)
    1 | 03m31s |    0.69917 |      2.6667 |      0.4444 |      2.6667 | 
bmacs: [0.5527777777777778, 0.5944444444444444, 0.5574074074074075, 0.5842592592592593, 0.5907407407407407, 0.5657407407407408, 0.5592592592592592, 0.524074074074074, 0.6185185185185186, 0.5185185185185185]
AVG: BMAC score: 0.5666 (+/- 0.0297)
    2 | 03m16s |    0.56657 |      2.5600 |      0.3964 |      0.6132 | 
bmacs: [0.6648148148148149, 0.6638888888888889, 0.6722222222222222, 0.6722222222222222, 0.6777777777777777, 0.7064814814814815, 0.6648148148148149, 0.674074074074074, 0.6990740740740741, 0.6453703703703703]

C:\Users\nicok\Miniconda3\envs\aml\lib\site-packages\sklearn\gaussian_process\gpr.py:480: ConvergenceWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-3.1483186e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 54, 'nit': 6, 'warnflag': 2}
  ConvergenceWarning)


bmacs: [0.6722222222222222, 0.662962962962963, 0.7037037037037037, 0.7111111111111111, 0.7092592592592593, 0.736111111111111, 0.6925925925925926, 0.6675925925925926, 0.7157407407407407, 0.6703703703703704]
AVG: BMAC score: 0.6942 (+/- 0.0236)
   21 | 03m52s |    0.69417 |      2.6241 |      0.3275 |      3.4418 | 


C:\Users\nicok\Miniconda3\envs\aml\lib\site-packages\sklearn\gaussian_process\gpr.py:480: ConvergenceWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([3.32331667e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 63, 'nit': 5, 'warnflag': 2}
  ConvergenceWarning)


bmacs: [0.6574074074074073, 0.6824074074074075, 0.7092592592592593, 0.6972222222222223, 0.7194444444444444, 0.7055555555555556, 0.6851851851851851, 0.6962962962962962, 0.7166666666666668, 0.6268518518518519]
AVG: BMAC score: 0.6896 (+/- 0.0272)
   22 | 03m21s |    0.68963 |      6.0000 |      1.0934 |      6.0000 | 
bmacs: [0.6620370370370371, 0.6805555555555557, 0.7074074074074073, 0.7175925925925926, 0.7453703703703703, 0.7148148148148148, 0.6703703703703704, 0.6944444444444445, 0.7287037037037037, 0.6490740740740741]
AVG: BMAC score: 0.6970 (+/- 0.0295)
   23 | 03m24s |    0.69704 |      2.9239 |      0.6162 |      2.7001 | 
bmacs: [0.6962962962962962, 0.6398148148148148, 0.6666666666666666, 0.6824074074074074, 0.7101851851851851, 0.7055555555555556, 0.6629629629629629, 0.6675925925925926, 0.6898148148148149, 0.6527777777777778]
AVG: BMAC score: 0.6774 (+/- 0.0220)
   24 | 04m35s |    0.67741 |      2.8702 |      0.1000 |      2.8101 | 
bmacs: [0.6592592592592593, 0.712037037037037,

...

bmacs: [0.6583333333333333, 0.6675925925925926, 0.686111111111111, 0.7101851851851851, 0.7027777777777778, 0.7388888888888889, 0.6888888888888888, 0.6777777777777777, 0.7351851851851853, 0.6814814814814815]
AVG: BMAC score: 0.6947 (+/- 0.0255)
  172 | 04m00s |    0.69472 |      2.5190 |      0.3621 |      4.2383 | 


C:\Users\nicok\Miniconda3\envs\aml\lib\site-packages\sklearn\gaussian_process\gpr.py:480: ConvergenceWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00056947]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 52, 'nit': 3, 'warnflag': 2}
  ConvergenceWarning)


bmacs: [0.6638888888888889, 0.6712962962962963, 0.6962962962962963, 0.7222222222222222, 0.7194444444444444, 0.7324074074074075, 0.6861111111111112, 0.6842592592592593, 0.7277777777777779, 0.6898148148148149]
AVG: BMAC score: 0.6994 (+/- 0.0232)
  173 | 03m43s |    0.69935 |      2.3428 |      0.4663 |      3.9007 | 
bmacs: [0.6685185185185185, 0.6712962962962963, 0.6972222222222223, 0.7175925925925926, 0.7203703703703703, 0.7361111111111112, 0.6814814814814815, 0.6833333333333332, 0.7296296296296297, 0.6851851851851851]
AVG: BMAC score: 0.6991 (+/- 0.0236)
  174 | 03m36s |    0.69907 |      2.3440 |      0.4890 |      3.8993 | 
bmacs: [0.6675925925925926, 0.674074074074074, 0.700925925925926, 0.7148148148148148, 0.7055555555555556, 0.7361111111111112, 0.674074074074074, 0.6916666666666668, 0.7287037037037037, 0.6851851851851851]
AVG: BMAC score: 0.6979 (+/- 0.0224)
  175 | 03m32s |    0.69787 |      2.3598 |      0.5354 |      3.9650 | 
bmacs: [0.6574074074074074, 0.6722222222222222, 0

C:\Users\nicok\Miniconda3\envs\aml\lib\site-packages\sklearn\gaussian_process\gpr.py:480: ConvergenceWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00017978]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 46, 'nit': 3, 'warnflag': 2}
  ConvergenceWarning)


KeyboardInterrupt: 

### Bayesian Optimization: Train SVM Gamma

In [ ]:
# Bayesian Optimization
from bayes_opt import BayesianOptimization

BO = BayesianOptimization(
        svm_cv,
        {'gamma': (0.00001, 0.01)},
      random_state=seed
)

BO.explore({'gamma': [1.0/n_features] })

gp_params = {'alpha': 1e-5}

BO.maximize(n_iter=200, **gp_params)
  
print(BO.res['max'])
print(BO.res['all'])